In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import random
import math

In [3]:
horizon = 50000

In [4]:
####### Simulating M/D/C conditions 
for C in range(1,6):
    mu = 1.0 ;
    abs_error = []
    rel_error = []
    q_error = []
    true_delay = []
    formula_delay = []
    for rho in np.arange(0.1,0.92,0.02): 
        print(rho,C)
        lamda = rho * mu * C
        
        ####### Generate arrivals for this trial #######
        arriv_hash = {x:[] for x in range(horizon+1)}
        arrivals = [0 for _ in range(horizon*(math.ceil(lamda)))]
        for i in range(1,horizon*(math.ceil(lamda))):
                        arrivals[i] = arrivals[i-1] + random.expovariate(lamda)
                        if arrivals[i] >= horizon:
                            break;
                        else:
                            arriv_hash[int(arrivals[i])].append(arrivals[i])
        arrivals.pop(0)
        ################################################
        
        queue = []
        finished = []
        servers = [['nan',0] for x in range(C)]
        for i in range(1,horizon):
             queue.extend(arriv_hash[i-1])
             for k in range(C):
                 if queue == [] and servers[k][0] != 'nan' and servers[k][1] <= i:
                    servers[k][0] = 'nan'
                 elif servers[k][0] =='nan' and queue != []:
                    servers[k][0] = queue[0]
                    servers[k][1] = 1/mu + queue[0]
                    queue.pop(0)
                    finished.append(1)
                 elif servers[k][1]<=i and queue != []:
                    if queue[0] >= servers[k][1]:
                        servers[k][0] = queue[0]
                        servers[k][1] = 1/mu + queue[0]
                        finished.append(servers[k][1]-queue[0])
                    else:
                        servers[k][0] = queue[0]
                        servers[k][1] = 1/mu + servers[k][1]
                        finished.append(servers[k][1]-queue[0])
                    queue.pop(0)

        ### Calculating M/D/C Delay Approximation            
        alpha = [1 for x in range(C)]
        for i in range(1,C):
            alpha[i] = 1 + (mu/lamda) * i * alpha[i-1]
        mmc_delay = lamda / ( ((C*mu-lamda)**2) * (alpha[C-1] + lamda /(C * mu - lamda)))
        mdc_delay = (mmc_delay / 2) * (1 + ( (1-rho) * (C-1) * (np.sqrt(4 + 5*C) - 2) ) / (16 * rho * C) ) + 1/mu
        abs_error.append(abs(mdc_delay-np.mean(finished)))
        rel_error.append(abs(mdc_delay-np.mean(finished))/np.mean(finished))
        q_error.append(abs(mdc_delay-np.mean(finished))/(np.mean(finished)-(1/mu)))
        true_delay.append(np.mean(finished)- (1/mu))
        formula_delay.append(mdc_delay- (1/mu))

    ######### Write Files ###########
    df = pd.DataFrame()
    df['rel_error'] = rel_error
    df['abs_error'] = abs_error
    df['q_rel_error'] = q_error
    df['true_delay'] = true_delay
    df['formula_delay'] = formula_delay
    df['rho'] = list(np.arange(0.1,0.92,0.02))
    df.to_csv(f'output/error_{C}.csv', index = False)

0.1 1
0.12000000000000001 1
0.14 1
0.16000000000000003 1
0.18000000000000002 1
0.2 1
0.22000000000000003 1
0.24000000000000002 1
0.26 1
0.28 1
0.30000000000000004 1
0.32000000000000006 1
0.3400000000000001 1
0.3600000000000001 1
0.38 1
0.4 1
0.42000000000000004 1
0.44000000000000006 1
0.4600000000000001 1
0.4800000000000001 1
0.5000000000000001 1
0.5200000000000001 1
0.54 1
0.56 1
0.5800000000000001 1
0.6000000000000001 1
0.6200000000000001 1
0.6400000000000001 1
0.66 1
0.68 1
0.7000000000000001 1
0.7200000000000001 1
0.7400000000000001 1
0.7600000000000001 1
0.7800000000000001 1
0.8000000000000002 1
0.8200000000000002 1
0.8400000000000001 1
0.8600000000000001 1
0.8800000000000001 1
0.9000000000000001 1
0.1 2
0.12000000000000001 2
0.14 2
0.16000000000000003 2
0.18000000000000002 2
0.2 2
0.22000000000000003 2
0.24000000000000002 2
0.26 2
0.28 2
0.30000000000000004 2
0.32000000000000006 2
0.3400000000000001 2
0.3600000000000001 2
0.38 2
0.4 2
0.42000000000000004 2
0.44000000000000006 2
0

In [29]:
servers

[[49998.10134096537, 49999.74147131459],
 [49998.150354880156, 49999.94336724357],
 [49998.415832896586, 49999.901214057114],
 [49998.79450502624, 49999.79450502624],
 ['nan', 49998.9862115753]]